In [1]:
import sys

sys.path.append('../data')
sys.path.append('..')
sys.path.append('../inference')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from ai_trainer import Personal_Trainer
import squat_separation as ss
import pickle
import os
import random as rnd
from collections import defaultdict
import classification
import utils as ut


%matplotlib inline


#Train Classifiers

In [7]:
pt = Personal_Trainer('NeckY')

try:
    pt.load_squats(os.path.join('../data/data_sets','multipleClass4.p'))
    ut.print_success('Training data loaded')
    try:
        pt.set_classifiers(classification.train_classifiers(pt))
        ut.print_success('Classifiers trained')
    except Exception as e:
        ut.print_failure('Could not train classifiers' + str(e))
except Exception as e:
    ut.print_failure('Could not load training data:' + str(e))

[SUCCESS] ---------- > Training data loaded
[SUCCESS] ---------- > Classifiers trained


#Store Classifiers

In [8]:
classifiers = pt.get_classifiers()
pickle.dump(classifiers,open('trained_classifiers2.p','wb'))

In [2]:
pt = Personal_Trainer('NeckY')

try:
	pt.load_squats(os.path.join('../data/data_sets','multipleClass4.p'))
	ut.print_success('Training data loaded')
	try:
		classifiers = pickle.load(open(os.path.join('../inference/','trained_classifiers2.p'),'rb'))
		pt.set_classifiers(classifiers)
		ut.print_success('Classifiers trained')
	except Exception as e:
		ut.print_failure('Could not train classifiers' + str(e))
except Exception as e:
	ut.print_failure('Could not load training data:' + str(e))

[SUCCESS] ---------- > Training data loaded
[SUCCESS] ---------- > Classifiers trained


In [3]:
squats = pt.analyze_squats('../squatData.txt')

In [4]:
print len(squats)

6
